# 400.1 | Prompt Engineering For Developers

Notes and exercises based on the [ChatGPT Prompt Engineering For Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) course from DeepLearning.AI.

---

### 0. Validate Setup

Before you begin working on exercises in this section, make sure you are setup to work with the right LLM Providers by running the relevant code cells from the [LLM Setup](./400-00-llm-setup.ipynb) notebook.

---

### 1. Introduction 

This course was created when OpenAI was using version 0.28. In this notebook I hope to use the course for insights, but use it in conjunction with the OpenAI documentation to update the learnings for the latest OpenAI versions and practices. Quick study notes

1. LLM = Large Language Model. Understands Natural Language. Takes NLP inputs (prompts), returns NLP outputs (responses).
1. 2 types of LLM = Base LLM (predicts next word), Instruction Tuned LLM (follows instructions)
1. Base LLM = Default, also referred to as Foundation Models that are trained on large corpora of text data.
1. Instruction Tuned LLM = Fine-tuned version of base LLM using RLHF (Reinforcement Learning from Human Feedback) to follow instructions.
1. Generative AI Focus = Increasingly on Instruction Tuned LLMs, and using NLP to have AI execute tasks based on instruction prompts.

LLMs are smart in the sense of having a massive knowledge base and fast pattern-matching capabilities. But they lack the human cognitive ability to understand context, reason, and infer meaning. Instead, they are effectively reflecting the data they were trained on - which means instruction-tuned LLMs are only as effective as the clarity and specificity of instructions they are given.


---

### 2. Guidelines for Prompting

The chaper focuses on 2 main guidelines:
1. Write clear and concise instructions
1. Give the model time to think.

The course has its own sandbox environment for practice, but I want to try my own experiments here. 
- Refer to the [Setup notebook](400-00-llm-setup.ipynb) for code snippets for chat completion
- Refer to the [OpenAI API documentation](https://platform.openai.com/docs/quickstart?context=python) for quickstarts
- Refer to the [OpenAI Cookbook](https://cookbook.openai.com/) for richer examples

In [3]:
# Create an OpenAI client instance
from openai import OpenAI
client = OpenAI()

# Create a helper function to reuse across exercises
# So we can focus on prompt engineering more clearly
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt }]
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0, #degree of randomness
  )
  return completion.choices[0].message.content

# Prompting Principles
# 1. Write clear and specific instructions
# 2. Give the model time to "think"
# =====================================

In [ ]:

# Tactic: Use Delimiters to Guide Model
# =====================================

# Make sure cells are run in order to have client setup.

text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

# Craft Your Prompt
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

# Print Completion Response
print(get_completion(prompt))